# vector store 로딩

In [1]:
import os
os.environ['OPENAI_API_KEY'] = open('API_KEY', 'r').read()

In [2]:
from langchain_postgres import PGVector
from langchain_postgres.vectorstores import PGVector
from langchain_openai import OpenAIEmbeddings

connection=f"postgresql+psycopg://postgres@localhost:5432/my_vec_db"
collection_name = "kb_insure2"

vector_store = PGVector(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-large"),
    collection_name=collection_name,
    connection=connection,
    use_jsonb=True,
)

In [3]:
from langchain.document_loaders import PyPDFLoader

# 약관문서 샘플 : https://www.kbinsure.co.kr/CG802030002.ec
# 20240401 일자 버전.
# PDF 파일 로드
loader = PyPDFLoader("/Users/jaesolshin/Documents/GitHub/pg_test/data/20240401_15101_1.pdf")
document = loader.load()
document[0].page_content[:200] # 내용 추출

'KB개인상해보험'

In [4]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(document)
vector_store.add_documents(texts)

['4af78eeb-2e01-4277-8aec-5fe249497341',
 '89062092-19d3-428f-8590-9847beb1e71a',
 '340a9f58-067f-46ca-8df8-5739fdde18d7',
 'a51d0ab9-f874-449a-b061-63bc5f8f6ab9',
 '6836a4ba-7d9c-41ba-b6b6-81e6436b173f',
 '9888f88b-e925-4b28-bc85-a43a46f55737',
 'f2f67898-1662-4840-abca-992a52eaa5c7',
 'd26a91d8-c514-4fa2-919a-ee42a35fbfd1',
 'bc3d31fd-4037-4b98-827e-60b9655a0f54',
 '3c117efb-a666-47d7-b91d-110bd3dd8e58',
 '86756d8c-f3bc-4d9b-941f-d4cf9be42a39',
 'f0de6d30-3550-4b29-a59d-243107db177b',
 '87e5a113-682b-48e8-b094-14eb75d0b0e7',
 '9c4fb356-6b32-40d4-8391-5d710467754e',
 '987b12c0-533c-460e-a6a8-4b38a5e62f40',
 '038d0846-7f0b-46e5-9420-eee3aa23a9ef',
 'fc55d3d5-d7b3-485a-afcf-343982d146e1',
 'cffb23ee-bd98-497f-9d7c-721d4655327d',
 '62f39040-d0f2-4ad1-b7ed-9572a20002b0',
 '70279526-8eb9-47dc-809d-f6f6b2996ddd',
 'e2574b19-afd2-4026-b32d-aed0e0b34d80',
 '8f972911-367d-49df-9ca7-44f28ee51117',
 '0cb834f9-72cf-4367-bce7-5450d2822713',
 '146ad15e-aa54-48b4-a13f-c0f1fb6c1cca',
 'a9febddb-bc80-

# Multi-Query

## Multi-query 생성 및 문서 검색

MultiQueryRetriever 클래스를 사용해 주어진 질문과 연관된 추가 쿼리를 여러 개 생성하고, 각각의 쿼리를 이용해 문서를 검색합니다.

In [5]:
# 멀티 쿼리 생성
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = '보험금의 지급사유'

llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

# MultiQueryRetriever 설정
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm
)

# Set logging for the queries
import logging

# 생성되는 multi-query 확인
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

# query 에 대해 Multi-query 문서 검색 (질문과 관련 있는 문서 검색)
relevant_docs = retriever_from_llm.get_relevant_documents(query=question)
relevant_docs

# get_relevant_documents 대신 invoke 를 이용할 수도 있습니다.
# relevant_docs = retriever_from_llm.invoke(question)
# relevant_docs

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5881/506101021.py:22: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever_from_llm.get_relevant_documents(query=question)
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 보험금 지급을 결정하는 기준은 무엇인가요?', '2. 어떤 상황에서 보험금을 받을 수 있나요?', '3. 보험금 지급 조건에는 어떤 것들이 포함되나요?']


[Document(id='987b12c0-533c-460e-a6a8-4b38a5e62f40', metadata={'page': 14, 'source': '/Users/jaesolshin/Documents/GitHub/pg_test/data/20240401_15101_1.pdf'}, page_content='- 4 -\n 4. 기타 보험수익자가 보험금의 수령에 필요하여 제출하는 서류(사망보험금 지급시 피보험자의 법정상속인이 아닌자가 청구하는 경우 법정상속인의 확인서)② 제1항 제2호의 사고증명서는 의료법 제3조(의료기관)에서 규정한 국내의 병원이나 의원 또는 국외의 의료관련법에서 정한 의료기관에서 발급한 것이어야 합니다.제8조(보험금의 지급절차) ① 회사는 제7조(보험금의 청구)에서 정한 서류를 접수한 때에는 접수증을 드리고 휴대전화 문자메시지 또는 전자우편 등으로도 송부하며, 그 서류를 접수한 날부터 3영업일 이내에 보험금을 지급합니다.② 회사가 보험금 지급사유를 조사․확인하기 위해 필요한 기간이 제1항의 지급기일을 초과할 것이 명백히 예상되는 경우에는 그 구체적인 사유와 지급예정일 및 보험금 가지급제도(회사가 추정하는 보험금의 50% 이내를 지급)에 대하여 피보험자 또는 보험수익자에게 즉시 통지합니다. 다만, 지급예정일은 다음 각 호의 어느 하나에 해당하는 경우를 제외하고는 제7조(보험금의 청구)에서 정한 서류를 접수한 날부터 30영업일 이내에서 정합니다. 1. 소송제기 2. 분쟁조정 신청【분쟁조정 신청】분쟁조정 신청은 이 약관의「분쟁의 조정」조항에 따르며 분쟁조정 신청 대상기관은 금융감독원의 금융분쟁조정위원회를 말합니다. 3. 수사기관의 조사  4. 해외에서 발생한 보험사고에 대한 조사 5. 제6항에 따른 회사의 조사요청에 대한 동의 거부 등 계약자, 피보험자 또는 보험수익자의 책임있는 사유로 보험금 지급사유의 조사와 확인이 지연되는 경우 6. 제4조(보험금 지급에 관한 세부규정) 제6항에 따라 보험금 지급사유에 대해 제3자의 의견에 따르기로 한 경우③ 제

## LangChain 을 이용한 Multi-Query 답변 생성

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever


# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)


question = '보험금의 지급사유'

llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm
)

# 각각의 문서를 하나의 문자열로 join 합니다.
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

#  LangChain 을 생성합니다.
# retriever_from_llm 을 이용해 얻은 문서들을 context 로 하는 LangChain 을 생성해 최종 답변을 얻습니다.
chain = (
    {'context': retriever_from_llm | format_docs, 'question': RunnablePassthrough()}
    | ChatPromptTemplate.from_template(template)
    | llm
    | StrOutputParser()
)

# Run
response = chain.invoke(question)
response

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 보험금 지급을 결정하는 기준은 무엇인가요?', '2. 어떤 상황에서 보험금을 받을 수 있나요?', '3. 보험금 지급 조건에는 어떤 것들이 포함되나요?']


'보험금의 지급사유는 다음과 같습니다:\n\n1. 보험기간 중에 상해의 직접결과로써 사망한 경우(질병으로 인한 사망은 제외합니다): 사망보험금이 지급됩니다.\n\n2. 보험기간 중 상해로 장해분류표에서 정한 각 장해지급률에 해당하는 장해상태가 되었을 때: 후유장해보험금이 지급됩니다. 후유장해보험금은 장해분류표에서 정한 지급률을 보험가입금액에 곱하여 산출한 금액으로 결정됩니다.\n\n이 외에도 특정 조건 하에서 발생한 사고나 상황에 대해 보험 약관에 따라 보험금이 지급될 수 있으며, 보험금 지급에 관한 세부 규정이나 지급하지 않는 사유 등은 해당 보험 약관에 명시되어 있습니다.'